In [ ]:
import os
import glob

import BreakJunctionUtilities as bju

import numpy as np

import matplotlib.pyplot as plt

## Load Data

In [ ]:
directory = 'INSERT_DIRECTORY' # Load in mat files created by the sIsGroupSave.m MATLAB script
files = glob.glob(os.path.join(directory, '*.mat'))

files_data = []
for file in files:
    files_data.append(bju.loadmat(file))
    print(file)

## Example Traces and Log Traces

Analyse a single repeat to demonstrate the process.

In [ ]:
# Extract the current and distance arrays
test = files_data[1]
distances = np.array(test['struct']['distance']).T
currents = np.array(test['struct']['currents']).T

In [ ]:
# Filter out poor pulls
passed_traces = []
for i, trace in enumerate(currents):
    passed_traces.append(trace.mean() > 0.1)
passed_traces = np.array(passed_traces)

In [ ]:
currents = currents[passed_traces == 1, :]
distances = distances[passed_traces == 1, :]

In [ ]:
# Convert traces to a logarithmic scale
currents[currents < 0] = 0
log_curr = np.log((currents))
log_curr[log_curr < -8] = -8
ret = plt.hist2d(distances[:, :].flatten(), log_curr.flatten(), bins=121, cmax=3000)

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(6, 4), dpi=600)

for i in range(5):
    if i == 0:
        alpha = 1.0
    else:
        alpha = 0.3
    
    ans = axs[0].plot(distances.T[:, i], currents.T[:, i], alpha=alpha, c='#1f77b4')
    ans = axs[1].plot(distances.T[:, i], log_curr.T[:, i], alpha=alpha, c='#1f77b4')

x = distances.T[:200, 0]
y = log_curr.T[:, 0]

length = 500
x = distances.T[:length, 0]
y = log_curr.T[:length, 0]

A = np.vstack([x, np.ones(length)]).T
    
m, c = np.linalg.lstsq(A, y, rcond=None)[0]

fit_line, = axs[1].plot(x, m*x+c, color='red', dashes=[5, 2], linewidth=3)

axs[0].set(xlim=(0, 1E-9))
axs[0].set_xlabel('Distance / m', weight='bold')
axs[0].set_ylabel('Current / nA', weight='bold')

axs[1].set(xlim=(0, 1E-9))
axs[1].set_xlabel('Distance / m', weight='bold')
axs[1].set_ylabel('Log Current', weight='bold')

axs[0].annotate('(a)', xy=(0, 0), xytext=(0.0E-9, 10.8), weight='bold')
axs[1].annotate('(b)', xy=(0, 0), xytext=(0.0E-9, 3.4), weight='bold')

axs[0].tick_params(width=1.5)
for axis in ['top', 'right', 'bottom', 'left']:
    axs[0].spines[axis].set_linewidth(1.5)
axs[1].tick_params(width=1.5)
for axis in ['top', 'right', 'bottom', 'left']:
    axs[1].spines[axis].set_linewidth(1.5)

fig.tight_layout()

# fig.savefig("CurrLogCurr.png")

## Beta Histograms

Filter through all repeats to remove poor traces

In [ ]:
passed_traces_all = []
for data in files_data:
    traces = np.array(data['struct']['currents']).T
    passed_traces = []
    for trace in traces:
        passed_traces.append(trace.mean() > 0.1)
    passed_traces_all.append(np.array(passed_traces))

In [ ]:
betas = []
for i, data in enumerate(files_data):
    all_betas = np.array(data['struct']['betas'])
    passed_betas = all_betas[passed_traces_all[i]]
    betas.append(passed_betas)

Plot each beta distribution

In [ ]:
betas1, betas2, betas3 = betas

bin_edges = np.histogram_bin_edges(betas2, bins=100)
bin_mids = bin_edges[1:] - ((bin_edges[1] - bin_edges[0]) / 2)

counts = [np.histogram(x, bins=bin_edges)[0] for x in betas]
counts1, counts2, counts3 = counts

In [ ]:
fig, ax = plt.subplots(figsize=(6, 4), dpi=600)
h1 = ax.bar(bin_mids, counts1, width = np.diff(bin_edges), alpha=0.6)
h3 = ax.bar(bin_mids, counts3, width = np.diff(bin_edges), alpha=0.6)
h2 = ax.bar(bin_mids, counts2, width = np.diff(bin_edges), alpha=0.6)

ax.set_ylabel("Counts", weight='bold')
ax.set_xlabel("Beta / nm\N{SUPERSCRIPT MINUS}\N{SUPERSCRIPT ONE}", weight='bold')

ax.tick_params(width=1.5)
for axis in ['top', 'right', 'bottom', 'left']:
    ax.spines[axis].set_linewidth(1.5)
    
# fig.savefig("App Beta Hists.png")

### ANOVA Test

Test the null hypothesis before merging repeats

In [ ]:
from scipy.stats import f_oneway

F, p = f_oneway(betas1, betas2, betas3)
p

### Pool Datasets

Pool the three repeats before applying a gaussian fit

In [ ]:
all_betas = np.concatenate((betas1, betas2, betas3))

In [ ]:
bin_edges = np.histogram_bin_edges(all_betas, bins=100)
bin_mids = bin_edges[1:] - ((bin_edges[1] - bin_edges[0]) / 2)
counts = np.histogram(all_betas, bins=bin_edges)[0]

In [ ]:
from scipy.optimize import curve_fit

def gauss(x, *p):
    A, mu, sig = p
    return A * np.exp(-(x - mu)**2 / (2*sig**2))

coeff, var_mat = curve_fit(gauss, bin_mids, counts, p0=(1, 0, 1))
fit_vals = gauss(bin_mids, *coeff)

In [ ]:
fig, ax = plt.subplots(figsize=(6, 4), dpi=600)
res = ax.bar(bin_mids, counts, width=np.diff(bin_edges))
ax.plot(bin_mids, fit_vals, color='red', dashes=[5, 2], linewidth=3)

ax.set_xlabel('Beta / nm\N{SUPERSCRIPT MINUS}\N{SUPERSCRIPT ONE}', weight='bold')
ax.set_ylabel('Counts', weight='bold')

ax.tick_params(width=1.5)
for axis in ['top', 'right', 'bottom', 'left']:
    ax.spines[axis].set_linewidth(1.5)

print("Beta = {:.3f} \u00B1 {:.3f} / nm\N{SUPERSCRIPT MINUS}\N{SUPERSCRIPT ONE}".format(coeff[1], coeff[2]))

# fig.savefig('Betas.png')

In [ ]:
## Kolmogorov-Smirnov test
from scipy.stats import kstest

res = kstest(counts, fit_vals)
res.pvalue # goodness of fit

In [ ]:
# Plot a calibration curve
x = np.linspace(0, 2E9, 1000)
y = (np.log(3E-9) - np.log(77.4809E-6 * 0.1)) / (-x)

plt.plot(x, y)